In [10]:
import os
from langchain_openai import OpenAIEmbeddings
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from google.cloud import texttospeech_v1 as tts
import re
from dotenv import load_dotenv

load_dotenv()

# 테스트 스크립트
script = """이제 최초의 발표하는 인공지능 모델인 오인용에 대해 소개하겠습니다. 이 프로젝트는 중요한 내용을 중심으로 발표용 대본을 생성하는 기능을 가지고 있습니다. AI 음성 합성을 활용하여 자동으로 발표할 수 있는 기능을 제공함으로써, 기업, 연구자, 학생 등이 보다 효율적으로 정보를 활용할 수 있게 됩니다. 이 시스템은 일관된 발표 퀄리티를 유지하며, 발표에 드는 시간을 단축하는 기대 효과를 가지고 있습니다.

다음은 프로젝트 기획에 대해 설명드리겠습니다. 이 슬라이드에서는 프로젝트의 전반적인 기획 방향과 목표를 간략히 정리하겠습니다. 프로젝트의 기획은 체계적인 진행을 위해 매우 중요합니다."""

# 강조 키워드
input_keywords = ["인공지능"]

# 문장 분리 및 단어 토큰화
sentences = re.split(r'(?<=[.?!])\s+', script.strip())
tokenized_sentences = [re.findall(r'\w+', s) for s in sentences]
unique_words = sorted(set(word for sent in tokenized_sentences for word in sent))

# 임베딩
embedder = OpenAIEmbeddings()
word_embeddings = embedder.embed_documents(unique_words)
keyword_embeddings = [embedder.embed_query(k) for k in input_keywords]

# 유사도 계산
similarities = {}
for word, w_emb in zip(unique_words, word_embeddings):
    sims = [cosine_similarity([w_emb], [k_emb])[0][0] for k_emb in keyword_embeddings]
    similarities[word] = max(sims)

# 상위 3개 강조 단어 선택
top_emphasized_words = {word for word, _ in sorted(similarities.items(), key=lambda x: x[1], reverse=True)[:3]}

# SSML 생성
ssml_outputs = []
for idx, sent in enumerate(sentences):
    words = re.split(r'(\W+)', sent)
    processed = [
        f'<emphasis level="strong">{w}</emphasis>' if w in top_emphasized_words else w
        for w in words
    ]
    ssml = f"<speak>{''.join(processed).strip()}</speak>"
    ssml_outputs.append((ssml, f"output_{idx}.wav"))

# Google TTS 요청
client = tts.TextToSpeechClient()
voice = tts.VoiceSelectionParams(language_code="ko-KR", name="ko-KR-Standard-B")
audio_config = tts.AudioConfig(audio_encoding=tts.AudioEncoding.LINEAR16)

for ssml, filename in ssml_outputs:
    print(f"[SSML 미리보기]\n{ssml}\n→ {filename}\n")
    response = client.synthesize_speech(
        input=tts.SynthesisInput(ssml=ssml),
        voice=voice,
        audio_config=audio_config
    )
    with open(filename, "wb") as out:
        out.write(response.audio_content)
        print(f"{filename} 저장 완료\n")


[SSML 미리보기]
<speak>이제 최초의 발표하는 <emphasis level="strong">인공지능</emphasis> 모델인 오인용에 대해 소개하겠습니다.</speak>
→ output_0.wav

output_0.wav 저장 완료

[SSML 미리보기]
<speak>이 프로젝트는 중요한 내용을 중심으로 발표용 대본을 생성하는 기능을 가지고 있습니다.</speak>
→ output_1.wav

output_1.wav 저장 완료

[SSML 미리보기]
<speak><emphasis level="strong">AI</emphasis> 음성 합성을 활용하여 자동으로 발표할 수 있는 기능을 제공함으로써, 기업, 연구자, 학생 등이 보다 효율적으로 정보를 활용할 수 있게 됩니다.</speak>
→ output_2.wav

output_2.wav 저장 완료

[SSML 미리보기]
<speak>이 시스템은 일관된 발표 퀄리티를 유지하며, 발표에 드는 시간을 단축하는 기대 효과를 가지고 있습니다.</speak>
→ output_3.wav

output_3.wav 저장 완료

[SSML 미리보기]
<speak>다음은 프로젝트 기획에 대해 설명드리겠습니다.</speak>
→ output_4.wav

output_4.wav 저장 완료

[SSML 미리보기]
<speak>이 슬라이드에서는 프로젝트의 전반적인 <emphasis level="strong">기획</emphasis> 방향과 목표를 간략히 정리하겠습니다.</speak>
→ output_5.wav

output_5.wav 저장 완료

[SSML 미리보기]
<speak>프로젝트의 기획은 체계적인 진행을 위해 매우 중요합니다.</speak>
→ output_6.wav

output_6.wav 저장 완료

